In [1]:
import json
import os
import glob
import shutil

In [2]:
def convert_multiple_json_to_yolo(annotations_dir, images_dir, output_dir, class_mapping=None):
    """
    Chuyển đổi nhiều file JSON annotation thành định dạng YOLO, hỗ trợ kiểu Supervisely (objects/classTitle/points.exterior).
    """
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    
    json_files = glob.glob(os.path.join(annotations_dir, '*.json'))
    if len(json_files) == 0:
        print(f"Không tìm thấy file JSON nào trong {annotations_dir}")
        return
    
    print(f"Tìm thấy {len(json_files)} file JSON annotation")
    
    # Build class_mapping nếu chưa có
    if class_mapping is None:
        print("Đang xây dựng class mapping từ annotation...")
        class_names = set()
        for json_file in json_files:
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                # Supervisely format
                if 'objects' in data:
                    for obj in data['objects']:
                        if 'classTitle' in obj:
                            class_names.add(obj['classTitle'])
                # Các format khác nếu cần
                elif 'shapes' in data:
                    for shape in data['shapes']:
                        class_names.add(shape['label'])
                elif 'annotations' in data:
                    for ann in data['annotations']:
                        if 'category_name' in ann:
                            class_names.add(ann['category_name'])
                        elif 'category_id' in ann and 'categories' in data:
                            for cat in data['categories']:
                                if cat['id'] == ann['category_id']:
                                    class_names.add(cat['name'])
                                    break
            except Exception as e:
                print(f"Lỗi khi đọc file {json_file}: {e}")
        class_mapping = {name: idx for idx, name in enumerate(sorted(class_names))}
        if len(class_mapping) == 0:
            print("Không thể tạo class mapping từ annotation. Vui lòng cung cấp class_mapping.")
            return
    
    # Lưu class_mapping ra file
    with open(os.path.join(output_dir, 'classes.txt'), 'w', encoding='utf-8') as f:
        for class_name, class_id in sorted(class_mapping.items(), key=lambda x: x[1]):
            f.write(f"{class_name}\n")
    print(f"Đã lưu {len(class_mapping)} lớp vào classes.txt")
    
    processed_files = 0
    for json_file in json_files:
        try:
            base_name = os.path.basename(json_file)
            if '.jpg.json' in base_name:
                img_name = base_name.replace('.json', '')
            elif '.png.json' in base_name:
                img_name = base_name.replace('.json', '')
            else:
                img_name = os.path.splitext(base_name)[0] + '.jpg'
            src_img_path = os.path.join(images_dir, img_name)
            dst_img_path = os.path.join(output_dir, 'images', img_name)
            img_base_name = os.path.splitext(img_name)[0]
            label_file = os.path.join(output_dir, 'labels', img_base_name + '.txt')
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            if os.path.exists(src_img_path):
                shutil.copy(src_img_path, dst_img_path)
            else:
                print(f"Không tìm thấy hình ảnh {src_img_path}")
                continue
            # Lấy kích thước ảnh
            if 'imageWidth' in data and 'imageHeight' in data:
                img_width = data['imageWidth']
                img_height = data['imageHeight']
            elif 'width' in data and 'height' in data:
                img_width = data['width']
                img_height = data['height']
            elif 'size' in data and 'width' in data['size'] and 'height' in data['size']:
                img_width = data['size']['width']
                img_height = data['size']['height']
            else:
                from PIL import Image
                try:
                    with Image.open(src_img_path) as img:
                        img_width, img_height = img.size
                except:
                    print(f"Không thể đọc kích thước hình ảnh {src_img_path}")
                    continue
            # Tạo file label YOLO
            with open(label_file, 'w') as f:
                # Supervisely format
                if 'objects' in data:
                    for obj in data['objects']:
                        label = obj['classTitle']
                        points = obj['points']['exterior']
                        if len(points) != 2:
                            continue
                        x1, y1 = points[0]
                        x2, y2 = points[1]
                        x_min = min(x1, x2)
                        y_min = min(y1, y2)
                        x_max = max(x1, x2)
                        y_max = max(y1, y2)
                        width = x_max - x_min
                        height = y_max - y_min
                        x_center = x_min + width / 2
                        y_center = y_min + height / 2
                        x_center /= img_width
                        y_center /= img_height
                        width /= img_width
                        height /= img_height
                        class_id = class_mapping.get(label, 0)
                        f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
                # Các format khác (labelme, coco) nếu cần
                elif 'shapes' in data:
                    for shape in data['shapes']:
                        if shape['shape_type'] == 'rectangle':
                            label = shape['label']
                            points = shape['points']
                            x1, y1 = points[0]
                            x2, y2 = points[1]
                            x_min = min(x1, x2)
                            y_min = min(y1, y2)
                            x_max = max(x1, x2)
                            y_max = max(y1, y2)
                            width = x_max - x_min
                            height = y_max - y_min
                            x_center = x_min + width / 2
                            y_center = y_min + height / 2
                            x_center /= img_width
                            y_center /= img_height
                            width /= img_width
                            height /= img_height
                            class_id = class_mapping.get(label, 0)
                            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
                elif 'annotations' in data:
                    for ann in data['annotations']:
                        if 'bbox' in ann:
                            if 'category_id' in ann:
                                class_id = ann['category_id']
                            elif 'category_name' in ann:
                                class_id = class_mapping.get(ann['category_name'], 0)
                            else:
                                continue
                            bbox = ann['bbox']  # [x, y, width, height]
                            x_center = (bbox[0] + bbox[2]/2) / img_width
                            y_center = (bbox[1] + bbox[3]/2) / img_height
                            width = bbox[2] / img_width
                            height = bbox[3] / img_height
                            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
            processed_files += 1
        except Exception as e:
            print(f"Lỗi khi xử lý file {json_file}: {e}")
    print(f"Đã xử lý {processed_files}/{len(json_files)} file annotation")
    print(f"Dữ liệu đã được chuyển đổi và lưu vào {output_dir}")
    create_data_yaml(output_dir, class_mapping)

def create_data_yaml(output_dir, class_mapping):
    """Tạo file data.yaml cho YOLOv8"""
    class_names = [name for name, _ in sorted(class_mapping.items(), key=lambda x: x[1])]
    yaml_content = {
        'path': os.path.abspath(output_dir),
        'train': './images',
        'val': './images',
        'nc': len(class_names),
        'names': class_names
    }
    yaml_file = os.path.join(output_dir, 'data.yaml')
    import yaml
    with open(yaml_file, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)
    print(f"Đã tạo file cấu hình YAML: {yaml_file}")

# Các đường dẫn
annotations_dir = 'C:/Users/minhk/Downloads/IoT/dataset/Supermarket_shelves/annotations'
images_dir = 'C:/Users/minhk/Downloads/IoT/dataset/Supermarket_shelves/images'
output_dir = 'C:/Users/minhk/Downloads/IoT/model/dataset'

# Chạy hàm chuyển đổi
convert_multiple_json_to_yolo(annotations_dir, images_dir, output_dir)

Tìm thấy 45 file JSON annotation
Đang xây dựng class mapping từ annotation...
Đã lưu 2 lớp vào classes.txt
Đã xử lý 45/45 file annotation
Dữ liệu đã được chuyển đổi và lưu vào C:/Users/minhk/Downloads/IoT/model/dataset
Đã tạo file cấu hình YAML: C:/Users/minhk/Downloads/IoT/model/dataset\data.yaml


In [3]:
from ultralytics import YOLO

# Tạo file cấu hình YAML
yaml_file = 'C:/Users/minhk/Downloads/IoT/model/dataset/data.yaml'

# Khởi tạo mô hình (có thể chọn kích thước phù hợp: n, s, m, l, x)
model = YOLO('yolov8n.pt')  # load pretrained model

# Huấn luyện mô hình
results = model.train(
    data=yaml_file,
    epochs=50,
    imgsz=640,
    batch=16,
    patience=20
)

Ultralytics 8.3.126  Python-3.9.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/minhk/Downloads/IoT/model/dataset/data.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspec

train: Scanning C:\Users\minhk\Downloads\IoT\model\dataset\labels.cache... 45 images, 0 backgrounds, 1 corrupt: 100%|██

train: C:\Users\minhk\Downloads\IoT\model\dataset\images\005.jpg: 1 duplicate labels removed
train: C:\Users\minhk\Downloads\IoT\model\dataset\images\009.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.3022      1.3223      1.0841      1.0744      1.0623      1.0945      1.0893      1.1579      1.0835      1.2695      1.3175       1.113      1.3259       1.207       1.029      1.0884      1.3348      1.0344      1.3302      1.3182      1.3013        1.33      1.3374      1.0893      1.0893      1.0747
      1.0867      1.2417      1.0791      1.3228      1.0311      1.3339      1.3348      1.1824      1.3214      1.3235      1.2944       1.205      1.1221      1.2463      1.1656      1.0813      1.3266      1.3296      1.0859        1.33      1.0919      1.1205      1.0911      1.0887      1.0792      1.3214
      1.3395]
train: C:\Users\minhk\Downloads\IoT\model\dataset\images\018.jpg: 1 duplicate labels removed
train: C:\Users\minhk\Downloads\Io

val: Fast image access  (ping: 0.10.0 ms, read: 188.269.6 MB/s, size: 3609.6 KB)


val: Scanning C:\Users\minhk\Downloads\IoT\model\dataset\labels.cache... 45 images, 0 backgrounds, 1 corrupt: 100%|████

...

      50/50      5.42G      1.445     0.9938      0.985       3121        640: 100%|██████████| 3/3 [00:14<00:00,  4.87
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:15<0

                   all         44      11447      0.736      0.486      0.574       0.35



50 epochs completed in 0.911 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.2MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.126  Python-3.9.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<0


                   all         44      11447      0.735      0.486      0.574      0.349
                 Price         40       1676      0.705      0.369      0.442      0.246
               Product         44       9771      0.766      0.603      0.705      0.453
Speed: 1.1ms preprocess, 8.7ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to runs\detect\train2


In [5]:
from ultralytics import YOLO
import cv2
import numpy as np
from collections import Counter
import os
from glob import glob

# Đường dẫn tới model đã huấn luyện
MODEL_PATH = 'C:/Users/minhk/Downloads/IoT/runs/detect/train2/weights/best.pt'  # Đổi thành đường dẫn của bạn
OUTPUT_DIR = 'C:/Users/minhk/Downloads/IoT/output'

# Tải model
model = YOLO(MODEL_PATH)

def count_objects(image_path, conf_threshold=0.5):
    """
    Phát hiện và đếm đối tượng trên ảnh.
    """
    results = model(image_path)
    detections = results[0]
    boxes = detections.boxes
    class_counts = Counter()
    for box in boxes:
        class_id = int(box.cls[0])
        class_name = model.names[class_id]
        confidence = float(box.conf[0])
        if confidence > conf_threshold:
            class_counts[class_name] += 1
    return class_counts, detections

def visualize_detection(image_path, results, conf_threshold=0.5):
    """
    Vẽ bounding box và ghi nhãn lên ảnh.
    """
    img = cv2.imread(image_path)
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls[0])
        class_name = model.names[class_id]
        confidence = float(box.conf[0])
        if confidence > conf_threshold:
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"{class_name}: {confidence:.2f}"
            cv2.putText(img, label, (x1, max(0, y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    return img

def process_directory(input_dir, output_dir=OUTPUT_DIR, conf_threshold=0.5):
    """
    Xử lý toàn bộ ảnh trong thư mục, lưu ảnh kết quả ra output_dir.
    """
    os.makedirs(output_dir, exist_ok=True)
    image_files = glob(os.path.join(input_dir, '*.jpg')) + \
                  glob(os.path.join(input_dir, '*.png'))
    
    total_counts = Counter()
    for img_path in image_files:
        counts, detections = count_objects(img_path, conf_threshold)
        total_counts += counts
        output_img = visualize_detection(img_path, detections, conf_threshold)
        output_path = os.path.join(output_dir, os.path.basename(img_path))
        cv2.imwrite(output_path, output_img)
        print(f"Processed {img_path}: {dict(counts)}")
    
    print("\nTotal objects detected in all images:")
    for class_name, count in total_counts.items():
        print(f"{class_name}: {count}")

    return total_counts

if __name__ == "__main__":
    # Đổi đường dẫn thành thư mục chứa ảnh cần nhận diện
    input_directory = "C:/Users/minhk/Downloads/IoT/dataset/Supermarket_shelves/images"
    process_directory(input_directory)


image 1/1 C:\Users\minhk\Downloads\IoT\dataset\Supermarket_shelves\images\001.jpg: 448x640 268 Products, 49.5ms
Speed: 4.8ms preprocess, 49.5ms inference, 14.5ms postprocess per image at shape (1, 3, 448, 640)
Processed C:/Users/minhk/Downloads/IoT/dataset/Supermarket_shelves/images\001.jpg: {'Product': 240}

image 1/1 C:\Users\minhk\Downloads\IoT\dataset\Supermarket_shelves\images\002.jpg: 448x640 300 Products, 47.4ms
Speed: 4.9ms preprocess, 47.4ms inference, 4.0ms postprocess per image at shape (1, 3, 448, 640)
Processed C:/Users/minhk/Downloads/IoT/dataset/Supermarket_shelves/images\002.jpg: {'Product': 300}

image 1/1 C:\Users\minhk\Downloads\IoT\dataset\Supermarket_shelves\images\003.jpg: 448x640 18 Prices, 144 Products, 48.3ms
Speed: 6.7ms preprocess, 48.3ms inference, 4.0ms postprocess per image at shape (1, 3, 448, 640)
Processed C:/Users/minhk/Downloads/IoT/dataset/Supermarket_shelves/images\003.jpg: {'Product': 83, 'Price': 3}

image 1/1 C:\Users\minhk\Downloads\IoT\dataset

In [6]:
# Đánh giá mô hình trên tập kiểm tra
validation_results = model.val()
print(f"mAP@0.5: {validation_results.box.map50}")
print(f"mAP@0.5:0.95: {validation_results.box.map}")

Ultralytics 8.3.126  Python-3.9.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
val: Fast image access  (ping: 0.10.1 ms, read: 1560.5772.1 MB/s, size: 3255.4 KB)


val: Scanning C:\Users\minhk\Downloads\IoT\model\dataset\labels.cache... 45 images, 0 backgrounds, 1 corrupt: 100%|████

train: C:\Users\minhk\Downloads\IoT\model\dataset\images\005.jpg: 1 duplicate labels removed
train: C:\Users\minhk\Downloads\IoT\model\dataset\images\009.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.3022      1.3223      1.0841      1.0744      1.0623      1.0945      1.0893      1.1579      1.0835      1.2695      1.3175       1.113      1.3259       1.207       1.029      1.0884      1.3348      1.0344      1.3302      1.3182      1.3013        1.33      1.3374      1.0893      1.0893      1.0747
      1.0867      1.2417      1.0791      1.3228      1.0311      1.3339      1.3348      1.1824      1.3214      1.3235      1.2944       1.205      1.1221      1.2463      1.1656      1.0813      1.3266      1.3296      1.0859        1.33      1.0919      1.1205      1.0911      1.0887      1.0792      1.3214
      1.3395]
train: C:\Users\minhk\Downloads\IoT\model\dataset\images\018.jpg: 1 duplicate labels removed
train: C:\Users\minhk\Downloads\Io


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:17<0


                   all         44      11447      0.721      0.487      0.573       0.35
                 Price         40       1676       0.69      0.364       0.44      0.247
               Product         44       9771      0.751       0.61      0.705      0.452
Speed: 4.2ms preprocess, 113.2ms inference, 0.0ms loss, 49.5ms postprocess per image
Results saved to runs\detect\val
mAP@0.5: 0.5726136437958727
mAP@0.5:0.95: 0.3495889517341878
